In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import json

# Paths and constants
CHROMEDRIVER_PATH = 'chromedriver-win64/chromedriver.exe'
URL = 'https://www.maharashtramedicalcouncil.in/frmRmpList.aspx'
DROPDOWN_VALUE = "1"

def fetch_registration_data(numbers_to_insert):
    # Set up the Chrome WebDriver
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service)

    # Open the webpage
    driver.get(URL)

    # Wait until the dropdown element is present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
    )

    # Select dropdown value
    dropdown = Select(driver.find_element(By.NAME, 'ctl00$ContentPlaceHolder1$ddlSearch'))
    dropdown.select_by_value(DROPDOWN_VALUE)

    # Enter the numbers into the search field
    input_search_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtSearch')
    input_search_field.send_keys(numbers_to_insert)

    # Extract the text from the span element
    span_element = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_lblDisChar')
    extracted_text = span_element.text

    # Locate the second input field and send the extracted text
    input_disp_char_field = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_txtDispChar')
    input_disp_char_field.send_keys(extracted_text)

    # Click the search button
    search_button = driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_btnSearch')
    search_button.click()

    # Wait until the table is present
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'ctl00_ContentPlaceHolder1_gvList'))
    )

    # Extract the table rows
    rows = driver.find_elements(By.XPATH, "//table[@id='ctl00_ContentPlaceHolder1_gvList']//tr")

    # Prepare a list to store the extracted data
    data = []

    # Loop through the rows, excluding the header row
    for row in rows[1:]:
        # Extract the Name, Qualification, Registration No., Registration Date, and Valid Upto Date
        name = row.find_element(By.XPATH, ".//span[contains(@id, 'txtName')]").text
        qualification = row.find_element(By.XPATH, ".//span[contains(@id, 'txtQualification')]").text
        registration_no = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregNo')]").text
        registration_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtregdt')]").text
        valid_upto_date = row.find_element(By.XPATH, ".//span[contains(@id, 'txtValidupto')]").text

        # Append the extracted data to the list
        data.append({
            'Name': name,
            'Qualification': qualification,
            'Registration No.': registration_no,
            'Registration Date': registration_date,
            'Valid Upto Date': valid_upto_date
        })

    # Close the browser after the task is done
    driver.quit()

    # Convert the list into JSON
    json_output = json.dumps(data, indent=4)
    
    return json_output

# Example usage with a changeable number
numbers_to_insert = "77790"
json_data = fetch_registration_data(numbers_to_insert)

# Print the JSON output
print(json_data)


[
    {
        "Name": "MEHTA SHREYA MANOJ",
        "Qualification": "M.B.B.S., M.D. (Med. & Thera.),",
        "Registration No.": "77790",
        "Registration Date": "05/10/1995",
        "Valid Upto Date": "28/02/2027"
    }
]


: 